In [ ]:
# LIBRERIAS NECESARIAS 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pyodbc
from dotenv import dotenv_values
import psycopg2 as pg2    # Conectores para Postgres
import getpass  # Para obtener el usuario del sistema operativo

# Mostrar el DataFrame resultante
import ace_tools_open as tools

import uuid  # Importar la librería uuid

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category= FutureWarning)

secrets = dotenv_values(".env")   # Connection String from .env
folder = secrets["FOLDER_DATOS"]

import matplotlib.pyplot as plt

In [ ]:
id_proveedor = '189'
name= '189_BODEGAS_LOPEZ'
process_id ='747e5b07-82ab-4c4a-8dd6-17b1d20d30f2'
algoritmo = '189_BODEGAS_LOPEZ_ALGO_05'

# Recuperar Historial de Ventas
df_ventas = pd.read_csv(f'{folder}/{name}_Ventas.csv')
df_ventas['Codigo_Articulo']= df_ventas['Codigo_Articulo'].astype(int)
df_ventas['Sucursal']= df_ventas['Sucursal'].astype(int)
df_ventas['Fecha']= pd.to_datetime(df_ventas['Fecha'])

# Recuperar Maestro de Artículos
articulos = pd.read_csv(f'{folder}/{name}_Articulos.csv')
#articulos.head()

# Recuperar Maestro de Artículos
#df_validacion = pd.read_csv(f'{folder}/{algoritmo}_Datos_Validacion.csv')
#articulos.head()

# Recuperando Forecast Calculado
df_forecast = pd.read_csv(f'{folder}/{algoritmo}_Solicitudes_Compra.csv')
print(f"-> Datos Recuperados del CACHE: {id_proveedor}, Label: {name}")


In [ ]:
df_forecast['GRAFICO'] = None

In [ ]:
df_forecast.info()

In [ ]:
# Filtrar df_ventas para conservar solo los últimos 45 días
fecha_maxima = df_ventas["Fecha"].max()
df_ventas = df_ventas[df_ventas["Fecha"] >= (fecha_maxima - pd.Timedelta(days=56))]

In [ ]:
# DEFINIR EL GRAFICO EJEMPLO
articulo = 44450
sucursal = 1

df = df_ventas.copy()
df_filtrado = df[(df["Codigo_Articulo"] == articulo) & (df["Sucursal"] == sucursal)]

# df_validacion_filtrado = df_validacion[(df_validacion["Codigo_Articulo"] == articulo) & (df_validacion["Sucursal"] == sucursal)]

fig, ax = plt.subplots(
    figsize=(8, 6), nrows= 2, ncols= 2
)
fig.suptitle(f"Demanda Articulo {articulo} - Sucursal {sucursal}")
current_ax = 0
#Bucle para Llenar los gráficos
colors =["red", "blue", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

# Ventas Diarias
df_filtrado["Media_Movil"] = df_filtrado["Unidades"].rolling(window=7).mean()

# Ventas Diarias
ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Unidades"], marker="o", linestyle="-", label="Ventas", color=colors[0])
ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Media_Movil"], linestyle="--", label="Media Móvil (7 días)", color="black")
ax[0, 0].set_title("Ventas Diarias")
ax[0, 0].legend()
ax[0, 0].set_xlabel("Fecha")
ax[0, 0].set_ylabel("Unidades")
ax[0, 0].tick_params(axis='x', rotation=45)

# Ventas Semanales
df_filtrado["Semana"] = df_filtrado["Fecha"].dt.to_period("W").astype(str)
df_semanal = df_filtrado.groupby("Semana")["Unidades"].sum().reset_index()
df_semanal["Semana_Num"] = df_filtrado.groupby("Semana")["Fecha"].min().reset_index()["Fecha"].dt.isocalendar().week.astype(int)
df_semanal["Media_Movil"] = df_semanal["Unidades"].rolling(window=7).mean()

# Histograma de ventas semanales
ax[0, 1].bar(df_semanal["Semana_Num"], df_semanal["Unidades"], color=[colors[1],colors[2], colors[3], colors[4], colors[5]], alpha=0.7)

ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Media_Movil"], linestyle="--", label="Media Móvil (7 días)", color="black")
ax[0, 1].set_xlabel("Semana del Año")
ax[0, 1].set_ylabel("Unidades Vendidas")
ax[0, 1].set_title("Histograma de Ventas Semanales")
ax[0, 1].tick_params(axis='x', rotation=60)
ax[0, 1].grid(axis="y", linestyle="--", alpha=0.7)
# Estete es una forma de continuar
# Como podemos 


df = df_forecast.copy()
df_forecast_f = df[(df["Codigo_Articulo"] == articulo) & (df["Sucursal"] == sucursal)]

# Graficar el Forecast vs Ventas Reales en la tercera celda
labels = ["Forecast","Actual", "Anterior", "Año Ant"]
values = [df_forecast_f["Forecast"].iloc[0], df_forecast_f["ventas_last"].iloc[0], df_forecast_f["ventas_previous"].iloc[0], df_forecast_f["ventas_same_year"].iloc[0]]

ax[1, 0].bar(labels, values, color=[colors[2], colors[3], colors[4], colors[5]], alpha=0.7)
ax[1, 0].set_title("Forecast vs Ventas Anteriores")
ax[1, 0].set_ylabel("Unidades")
ax[1, 0].grid(axis="y", linestyle="--", alpha=0.7)


# Definir fechas de referencia
fecha_maxima = df_filtrado["Fecha"].max()
fecha_inicio_ultimos30 = fecha_maxima - pd.Timedelta(days=30)
fecha_inicio_previos30 = fecha_inicio_ultimos30 - pd.Timedelta(days=30)
fecha_inicio_anio_anterior = fecha_inicio_ultimos30 - pd.DateOffset(years=1)
fecha_fin_anio_anterior = fecha_inicio_previos30 - pd.DateOffset(years=1)

# Calcular ventas de los últimos 30 días
ventas_ultimos_30 = df_filtrado[(df_filtrado["Fecha"] > fecha_inicio_ultimos30)]["Unidades"].sum()

# Calcular ventas de los 30 días previos a los últimos 30 días
ventas_previos_30 = df_filtrado[
    (df_filtrado["Fecha"] > fecha_inicio_previos30) & (df_filtrado["Fecha"] <= fecha_inicio_ultimos30)
]["Unidades"].sum()

# Simulación de datos para las ventas del año anterior
df_filtrado_anio_anterior = df_filtrado.copy()
df_filtrado_anio_anterior["Fecha"] = df_filtrado_anio_anterior["Fecha"] - pd.DateOffset(years=1)

ventas_mismo_periodo_anio_anterior = df_filtrado_anio_anterior[
    (df_filtrado_anio_anterior["Fecha"] > fecha_inicio_anio_anterior) &
    (df_filtrado_anio_anterior["Fecha"] <= fecha_fin_anio_anterior)
]["Unidades"].sum()

# Datos para el histograma
labels = ["Últimos 30", "Anteriores 30", "Año anterior"]
values = [ventas_ultimos_30, ventas_previos_30, ventas_mismo_periodo_anio_anterior]

# Graficar el histograma en la celda [1,1]
ax[1, 1].bar(labels, values, color=[colors[0], colors[1], colors[2]], alpha=0.7)
ax[1, 1].set_title("Comparación de Ventas en 3 Períodos")
ax[1, 1].set_ylabel("Unidades Vendidas")
ax[1, 1].grid(axis="y", linestyle="--", alpha=0.7)

# Mostrar el gráfico
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajustar para no solapar con el título
plt.show()



In [ ]:
# Corregir el código asegurando que Semana_Num sea un entero válido para los ejes
df_semanal["Semana_Num"] = df_filtrado.groupby("Semana")["Fecha"].min().reset_index()["Fecha"].dt.isocalendar().week.astype(int)

# Graficar el histograma semanal con etiquetas compactas
plt.figure(figsize=(10, 5))
plt.bar(df_semanal["Semana_Num"], df_semanal["Unidades"], color='blue', alpha=0.7)
plt.xlabel("Semana del Año")
plt.ylabel("Unidades Vendidas")
plt.title("Histograma de Ventas Semanales")
plt.xticks(df_semanal["Semana_Num"], rotation=60)  # Rotación para mejor visibilidad
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Mostrar el gráfico
plt.show()



In [ ]:
import seaborn as sns

In [ ]:
articulo = 44450
sucursal = 1

df = df_ventas.copy()
df_filtrado = df[(df["Codigo_Articulo"] == articulo) & (df["Sucursal"] == sucursal)]

fig, ax = plt.subplots(
    figsize=(8, 6), nrows= 2, ncols= 2
)
fig.suptitle(f"Demanda Articulo {articulo} - Sucursal {sucursal}")
current_ax = 0
colors =["red", "blue", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

#Bucle para Llenar los gráficos
colors = ["red", "blue", "green", "orange"]

# Ventas Diarias
df_filtrado["Media_Movil"] = df_filtrado["Unidades"].rolling(window=7).mean()

ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Unidades"], marker="o", linestyle="-", label="Ventas", color=colors[0])
ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Media_Movil"], linestyle="--", label="Media Móvil (7 días)", color="black")
ax[0, 0].set_title("Ventas Diarias con Media Móvil")
ax[0, 0].legend()
ax[0, 0].set_xlabel("Fecha")
ax[0, 0].set_ylabel("Unidades")
ax[0, 0].tick_params(axis='x', rotation=45)
ax[0, 0].grid(axis="y", linestyle="--", alpha=0.7)


# Ventas Semanales
df_filtrado["Semana"] = df_filtrado["Fecha"].dt.to_period("W").astype(str)
df_semanal = df_filtrado.groupby("Semana")["Unidades"].sum().reset_index()
df_semanal["Semana_Num"] = df_filtrado.groupby("Semana")["Fecha"].min().reset_index()["Fecha"].dt.isocalendar().week.astype(int)

# Histograma de ventas semanales
ax[0, 1].bar(df_semanal["Semana_Num"], df_semanal["Unidades"], color=colors[2], alpha=0.7)
ax[0, 1].set_xlabel("Semana del Año")
ax[0, 1].set_ylabel("Unidades Vendidas")
ax[0, 1].set_title("Histograma de Ventas Semanales")
ax[0, 1].tick_params(axis='x', rotation=60)
ax[0, 1].grid(axis="y", linestyle="--", alpha=0.7)

# Ventas mensuales
df_filtrado["Mes"] = df_filtrado["Fecha"].dt.to_period("M").astype(str)
df_mensual = df_filtrado.groupby("Mes")["Unidades"].sum().reset_index()

ax[1,1].bar(df_mensual["Mes"], df_mensual["Unidades"], color="blue", alpha=0.7)
ax[1,1].set_xlabel("Mes")
ax[1,1].set_ylabel("Unidades Vendidas")
ax[1,1].set_title("Histograma de Ventas Mensuales")
ax[1,1].tick_params(axis='x', rotation=45)
ax[1,1].grid(axis="y", linestyle="--", alpha=0.7)

#Ventas por dia de semana
df_filtrado["Dia_Semana"] = df_filtrado["Fecha"].dt.day_name()
ventas_dia = df_filtrado.groupby("Dia_Semana")["Unidades"].sum().reindex(
    ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
)
sns.heatmap(ventas_dia.to_frame(), annot=True, cmap="coolwarm", fmt=".0f", linewidths=0.5, ax=ax[1,0])
ax[1,0].set_title("Ventas Totales por Día de la Semana")
ax[1,0].set_xlabel("")
ax[1,0].set_ylabel("")

# Mostrar el gráfico
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajustar para no solapar con el título
plt.show()


# articulo = 2020
# sucursal = 2

df = df_ventas.copy()
df_filtrado = df[(df["Codigo_Articulo"] == articulo) & (df["Sucursal"] == sucursal)]

fig, ax = plt.subplots(
    figsize=(8, 6), nrows= 2, ncols= 2
)
fig.suptitle(f"Demanda Articulo {articulo} - Sucursal {sucursal}")
current_ax = 0
colors =["red", "blue", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

#Bucle para Llenar los gráficos
colors = ["red", "blue", "green", "orange"]

# Ventas Diarias
df_filtrado["Media_Movil"] = df_filtrado["Unidades"].rolling(window=7).mean()

ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Unidades"], marker="o", linestyle="-", label="Ventas", color=colors[0])
ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Media_Movil"], linestyle="--", label="Media Móvil (7 días)", color="black")
ax[0, 0].set_title("Ventas Diarias con Media Móvil")
ax[0, 0].legend()
ax[0, 0].set_xlabel("Fecha")
ax[0, 0].set_ylabel("Unidades")
ax[0, 0].tick_params(axis='x', rotation=45)
ax[0, 0].grid(axis="y", linestyle="--", alpha=0.7)


# Ventas Semanales
df_filtrado["Semana"] = df_filtrado["Fecha"].dt.to_period("W").astype(str)
df_semanal = df_filtrado.groupby("Semana")["Unidades"].sum().reset_index()
df_semanal["Semana_Num"] = df_filtrado.groupby("Semana")["Fecha"].min().reset_index()["Fecha"].dt.isocalendar().week.astype(int)

# Histograma de ventas semanales
ax[0, 1].bar(df_semanal["Semana_Num"], df_semanal["Unidades"], color=colors[2], alpha=0.7)
ax[0, 1].set_xlabel("Semana del Año")
ax[0, 1].set_ylabel("Unidades Vendidas")
ax[0, 1].set_title("Histograma de Ventas Semanales")
ax[0, 1].tick_params(axis='x', rotation=60)
ax[0, 1].grid(axis="y", linestyle="--", alpha=0.7)

# Ventas mensuales
df_filtrado["Mes"] = df_filtrado["Fecha"].dt.to_period("M").astype(str)
df_mensual = df_filtrado.groupby("Mes")["Unidades"].sum().reset_index()

ax[1,1].bar(df_mensual["Mes"], df_mensual["Unidades"], color="blue", alpha=0.7)
ax[1,1].set_xlabel("Mes")
ax[1,1].set_ylabel("Unidades Vendidas")
ax[1,1].set_title("Histograma de Ventas Mensuales")
ax[1,1].tick_params(axis='x', rotation=45)
ax[1,1].grid(axis="y", linestyle="--", alpha=0.7)

#Ventas por dia de semana
df_filtrado["Dia_Semana"] = df_filtrado["Fecha"].dt.day_name()
ventas_dia = df_filtrado.groupby("Dia_Semana")["Unidades"].sum().reindex(
    ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
)
sns.heatmap(ventas_dia.to_frame(), annot=True, cmap="coolwarm", fmt=".0f", linewidths=0.5, ax=ax[1,0])
ax[1,0].set_title("Ventas Totales por Día de la Semana")
ax[1,0].set_xlabel("")
ax[1,0].set_ylabel("")

# Mostrar el gráfico
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajustar para no solapar con el título
plt.show()

In [ ]:
# RUTINA GENERADORA GLOBAL DE GRÄRICOS
import base64
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt

def generar_grafico_base64(df, articulo, sucursal, Forecast, Average, ventas_last, ventas_previous, ventas_same_year):
    df_filtrado = df[(df["Codigo_Articulo"] == articulo) & (df["Sucursal"] == sucursal)]

    # df_validacion_filtrado = df_validacion[(df_validacion["Codigo_Articulo"] == articulo) & (df_validacion["Sucursal"] == sucursal)]

    fig, ax = plt.subplots(
        figsize=(8, 6), nrows= 2, ncols= 2
    )
    fig.suptitle(f"Demanda Articulo {articulo} - Sucursal {sucursal}")
    current_ax = 0
    #Bucle para Llenar los gráficos
    colors =["red", "blue", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

    # Ventas Diarias
    df_filtrado["Media_Movil"] = df_filtrado["Unidades"].rolling(window=7).mean()

    # Ventas Diarias
    ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Unidades"], marker="o", linestyle="-", label="Ventas", color=colors[0])
    ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Media_Movil"], linestyle="--", label="Media Móvil (7 días)", color="black")
    ax[0, 0].set_title("Ventas Diarias")
    ax[0, 0].legend()
    ax[0, 0].set_xlabel("Fecha")
    ax[0, 0].set_ylabel("Unidades")
    ax[0, 0].tick_params(axis='x', rotation=45)

    # Ventas Semanales
    df_filtrado["Semana"] = df_filtrado["Fecha"].dt.to_period("W").astype(str)
    df_semanal = df_filtrado.groupby("Semana")["Unidades"].sum().reset_index()
    df_semanal["Semana_Num"] = df_filtrado.groupby("Semana")["Fecha"].min().reset_index()["Fecha"].dt.isocalendar().week.astype(int)
    df_semanal["Media_Movil"] = df_semanal["Unidades"].rolling(window=7).mean()

    # Histograma de ventas semanales
    ax[0, 1].bar(df_semanal["Semana_Num"], df_semanal["Unidades"], color=[colors[1],colors[2], colors[3], colors[4], colors[5]], alpha=0.7)

    ax[0, 0].plot(df_filtrado["Fecha"], df_filtrado["Media_Movil"], linestyle="--", label="Media Móvil (7 días)", color="black")
    ax[0, 1].set_xlabel("Semana del Año")
    ax[0, 1].set_ylabel("Unidades Vendidas")
    ax[0, 1].set_title("Histograma de Ventas Semanales")
    ax[0, 1].tick_params(axis='x', rotation=60)
    ax[0, 1].grid(axis="y", linestyle="--", alpha=0.7)
    # Estete es una forma de continuar
    # Como podemos 

    # Graficar el Forecast vs Ventas Reales en la tercera celda
    labels = ["Forecast","Actual", "Anterior", "Año Ant"]
    values = [Forecast, ventas_last, ventas_previous, ventas_same_year]

    ax[1, 0].bar(labels, values, color=[colors[2], colors[3], colors[4], colors[5]], alpha=0.7)
    ax[1, 0].set_title("Forecast vs Ventas Anteriores")
    ax[1, 0].set_ylabel("Unidades")
    ax[1, 0].grid(axis="y", linestyle="--", alpha=0.7)


    # Definir fechas de referencia
    fecha_maxima = df_filtrado["Fecha"].max()
    fecha_inicio_ultimos30 = fecha_maxima - pd.Timedelta(days=30)
    fecha_inicio_previos30 = fecha_inicio_ultimos30 - pd.Timedelta(days=30)
    fecha_inicio_anio_anterior = fecha_inicio_ultimos30 - pd.DateOffset(years=1)
    fecha_fin_anio_anterior = fecha_inicio_previos30 - pd.DateOffset(years=1)

    # Calcular ventas de los últimos 30 días
    ventas_ultimos_30 = df_filtrado[(df_filtrado["Fecha"] > fecha_inicio_ultimos30)]["Unidades"].sum()

    # Calcular ventas de los 30 días previos a los últimos 30 días
    ventas_previos_30 = df_filtrado[
        (df_filtrado["Fecha"] > fecha_inicio_previos30) & (df_filtrado["Fecha"] <= fecha_inicio_ultimos30)
    ]["Unidades"].sum()

    # Simulación de datos para las ventas del año anterior
    df_filtrado_anio_anterior = df_filtrado.copy()
    df_filtrado_anio_anterior["Fecha"] = df_filtrado_anio_anterior["Fecha"] - pd.DateOffset(years=1)

    ventas_mismo_periodo_anio_anterior = df_filtrado_anio_anterior[
        (df_filtrado_anio_anterior["Fecha"] > fecha_inicio_anio_anterior) &
        (df_filtrado_anio_anterior["Fecha"] <= fecha_fin_anio_anterior)
    ]["Unidades"].sum()

    # Datos para el histograma
    labels = ["Últimos 30", "Anteriores 30", "Año anterior", "Average"]
    values = [ventas_ultimos_30, ventas_previos_30, ventas_mismo_periodo_anio_anterior, Average]

    # Graficar el histograma en la celda [1,1]
    ax[1, 1].bar(labels, values, color=[colors[0], colors[1], colors[2]], alpha=0.7)
    ax[1, 1].set_title("Comparación de Ventas en 3 Períodos")
    ax[1, 1].set_ylabel("Unidades Vendidas")
    ax[1, 1].grid(axis="y", linestyle="--", alpha=0.7)

    # Mostrar el gráfico
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Ajustar para no solapar con el título

    # Guardar gráfico en base64
    buffer = BytesIO()
    plt.savefig(buffer, format="png")
    plt.close()
    
    return base64.b64encode(buffer.getvalue()).decode("utf-8")




In [ ]:
# ACORTAR EL FORECAST para que no tarde tanto

df_forecast = df_forecast[df_forecast["Sucursal"] <= 10]

In [ ]:
# Agregar la nueva columna de gráficos en df_forecast Iterando sobre todo el DATAFRAME

df_forecast["GRAFICO"] = df_forecast.apply(
    lambda row: generar_grafico_base64(df_ventas, row["Codigo_Articulo"], row["Sucursal"], row["Forecast"], row["Average"], row["ventas_last"], row["ventas_previous"], row["ventas_same_year"]) if not pd.isna(row["Codigo_Articulo"]) and not pd.isna(row["Sucursal"]) else None,
    axis=1
)


In [ ]:

# Mostrar la tabla con los gráficos en base64

tools.display_dataframe_to_user(name="Forecast con Gráficos", dataframe=df_forecast)

In [ ]:
df_forecast.to_csv(f'{folder}/{algoritmo}_Pronostico_Extendido.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from IPython.display import display, Image

# Función para decodificar y mostrar una imagen en Jupyter Notebook
def mostrar_grafico_base64(base64_string):
    if base64_string is None:
        print("No hay gráfico disponible.")
        return
    
    img_data = base64.b64decode(base64_string)
    buffer = BytesIO(img_data)
    
    # Mostrar la imagen en Jupyter Notebook
    display(Image(buffer.getvalue()))

# Seleccionar un gráfico aleatorio para visualizar
fila_ejemplo = df_forecast.dropna(subset=["GRAFICO"]).sample(2).iloc[0]
mostrar_grafico_base64(fila_ejemplo["GRAFICO"])
